In [1]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pyarrow.parquet as pq

C:\Users\ntuki\AppData\Roaming\Python\Python39\site-packages\networkx\utils\backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


In [3]:
# read monthly trip data and convert to dataframe
jan_trips = pq.read_table("fhvhv_tripdata_2021-01.parquet").to_pandas()
feb_trips = pq.read_table("fhvhv_tripdata_2021-02.parquet").to_pandas()
mar_trips = pq.read_table("fhvhv_tripdata_2021-03.parquet").to_pandas()
apr_trips = pq.read_table("fhvhv_tripdata_2021-04.parquet").to_pandas()
may_trips = pq.read_table("fhvhv_tripdata_2021-05.parquet").to_pandas()
jun_trips = pq.read_table("fhvhv_tripdata_2021-06.parquet").to_pandas()
jul_trips = pq.read_table("fhvhv_tripdata_2021-07.parquet").to_pandas()
aug_trips = pq.read_table("fhvhv_tripdata_2021-08.parquet").to_pandas()
sep_trips = pq.read_table("fhvhv_tripdata_2021-09.parquet").to_pandas()
oct_trips = pq.read_table("fhvhv_tripdata_2021-10.parquet").to_pandas()
nov_trips = pq.read_table("fhvhv_tripdata_2021-11.parquet").to_pandas()
dec_trips = pq.read_table("fhvhv_tripdata_2021-12.parquet").to_pandas()

In [3]:
# see what first 10 rows of dataframe looks like
jan_trips.head(10)

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02682,B02682,2021-01-01 00:28:09,2021-01-01 00:31:42,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,5.260,...,1.98,2.75,NaN,0.00,14.99,N,N,,N,N
1,HV0003,B02682,B02682,2021-01-01 00:45:56,2021-01-01 00:55:19,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,3.650,...,1.63,0.00,NaN,0.00,17.06,N,N,,N,N
2,HV0003,B02764,B02764,2021-01-01 00:21:15,2021-01-01 00:22:41,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,3.510,...,1.25,2.75,NaN,0.94,12.98,N,N,,N,N
3,HV0003,B02764,B02764,2021-01-01 00:39:12,2021-01-01 00:42:37,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,0.740,...,0.70,2.75,NaN,0.00,7.41,N,N,,N,N
4,HV0003,B02764,B02764,2021-01-01 00:46:11,2021-01-01 00:47:17,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,9.200,...,2.41,2.75,NaN,0.00,22.44,N,N,,N,N
5,HV0005,B02510,None,2021-01-01 00:04:00,NaT,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,9.725,...,2.49,2.75,NaN,0.00,28.90,N,N,N,N,N
6,HV0005,B02510,None,2021-01-01 00:40:06,NaT,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,2.469,...,2.22,0.00,NaN,0.00,15.01,N,N,N,N,N
7,HV0003,B02764,B02764,2021-01-01 00:10:36,2021-01-01 00:12:28,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,13.530,...,3.08,0.00,NaN,0.00,34.20,N,N,,N,N
8,HV0003,B02875,B02875,2021-01-01 00:21:17,2021-01-01 00:22:25,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,1.600,...,0.61,0.00,NaN,0.00,6.26,N,N,,N,N
9,HV0003,B02875,B02875,2021-01-01 00:36:57,2021-01-01 00:38:09,2021-01-01 00:40:12,2021-01-01 00:53:31,255,232,3.200,...,1.03,2.75,NaN,2.82,10.99,N,N,,N,N


In [4]:
# keep only rows with Uber trips, i.e. hvfhs_license_num HV0003
jan_trips = jan_trips.loc[jan_trips["hvfhs_license_num"]=="HV0003"]
feb_trips = feb_trips.loc[feb_trips["hvfhs_license_num"]=="HV0003"]
mar_trips = mar_trips.loc[mar_trips["hvfhs_license_num"]=="HV0003"]
apr_trips = apr_trips.loc[apr_trips["hvfhs_license_num"]=="HV0003"]
may_trips = may_trips.loc[may_trips["hvfhs_license_num"]=="HV0003"]
jun_trips = jun_trips.loc[jun_trips["hvfhs_license_num"]=="HV0003"]
jul_trips = jul_trips.loc[jul_trips["hvfhs_license_num"]=="HV0003"]
aug_trips = aug_trips.loc[aug_trips["hvfhs_license_num"]=="HV0003"]
sep_trips = sep_trips.loc[sep_trips["hvfhs_license_num"]=="HV0003"]
oct_trips = oct_trips.loc[oct_trips["hvfhs_license_num"]=="HV0003"]
nov_trips = nov_trips.loc[nov_trips["hvfhs_license_num"]=="HV0003"]
dec_trips = dec_trips.loc[dec_trips["hvfhs_license_num"]=="HV0003"]

In [5]:
# see what first 10 rows of current dataframe looks like
jan_trips.head(10)

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02682,B02682,2021-01-01 00:28:09,2021-01-01 00:31:42,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,5.26,...,1.98,2.75,NaN,0.00,14.99,N,N,,N,N
1,HV0003,B02682,B02682,2021-01-01 00:45:56,2021-01-01 00:55:19,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,3.65,...,1.63,0.00,NaN,0.00,17.06,N,N,,N,N
2,HV0003,B02764,B02764,2021-01-01 00:21:15,2021-01-01 00:22:41,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,3.51,...,1.25,2.75,NaN,0.94,12.98,N,N,,N,N
3,HV0003,B02764,B02764,2021-01-01 00:39:12,2021-01-01 00:42:37,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,0.74,...,0.70,2.75,NaN,0.00,7.41,N,N,,N,N
4,HV0003,B02764,B02764,2021-01-01 00:46:11,2021-01-01 00:47:17,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,9.20,...,2.41,2.75,NaN,0.00,22.44,N,N,,N,N
7,HV0003,B02764,B02764,2021-01-01 00:10:36,2021-01-01 00:12:28,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,13.53,...,3.08,0.00,NaN,0.00,34.20,N,N,,N,N
8,HV0003,B02875,B02875,2021-01-01 00:21:17,2021-01-01 00:22:25,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,1.60,...,0.61,0.00,NaN,0.00,6.26,N,N,,N,N
9,HV0003,B02875,B02875,2021-01-01 00:36:57,2021-01-01 00:38:09,2021-01-01 00:40:12,2021-01-01 00:53:31,255,232,3.20,...,1.03,2.75,NaN,2.82,10.99,N,N,,N,N
10,HV0003,B02875,B02875,2021-01-01 00:53:31,2021-01-01 00:56:21,2021-01-01 00:56:45,2021-01-01 01:17:42,232,198,5.74,...,1.52,2.75,NaN,0.00,17.61,N,N,,N,N
11,HV0003,B02835,B02835,2021-01-01 00:22:58,2021-01-01 00:27:01,2021-01-01 00:29:04,2021-01-01 00:36:27,113,48,1.80,...,0.73,2.75,NaN,0.00,6.12,N,N,,N,N


In [6]:
# see names of all dataframe columns
jan_trips.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [5]:
# list out irrelevant columns
irrelevant_cols = ["hvfhs_license_num", "dispatching_base_num", "originating_base_num", "request_datetime", "on_scene_datetime",
                   "trip_miles", "trip_time", "base_passenger_fare", "tolls", "bcf", "sales_tax", "congestion_surcharge",
                   "airport_fee", "tips", "driver_pay", "shared_request_flag", "shared_match_flag", "access_a_ride_flag",
                   "wav_request_flag", "wav_match_flag"]

In [6]:
# delete irrelevant columns
jan_trips = jan_trips.drop(columns=irrelevant_cols)
feb_trips = feb_trips.drop(columns=irrelevant_cols)
mar_trips = mar_trips.drop(columns=irrelevant_cols)
apr_trips = apr_trips.drop(columns=irrelevant_cols)
may_trips = may_trips.drop(columns=irrelevant_cols)
jun_trips = jun_trips.drop(columns=irrelevant_cols)
jul_trips = jul_trips.drop(columns=irrelevant_cols)
aug_trips = aug_trips.drop(columns=irrelevant_cols)
sep_trips = sep_trips.drop(columns=irrelevant_cols)
oct_trips = oct_trips.drop(columns=irrelevant_cols)
nov_trips = nov_trips.drop(columns=irrelevant_cols)
dec_trips = dec_trips.drop(columns=irrelevant_cols)

In [9]:
# see what current dataframe looks like
jan_trips.head(10)

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID
0,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166
1,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167
2,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142
3,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143
4,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78
7,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226
8,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255
9,2021-01-01 00:40:12,2021-01-01 00:53:31,255,232
10,2021-01-01 00:56:45,2021-01-01 01:17:42,232,198
11,2021-01-01 00:29:04,2021-01-01 00:36:27,113,48


In [7]:
# concatenate dataframes of each month together into one single dataframe
all_trips = pd.concat([jan_trips, feb_trips, mar_trips, apr_trips, may_trips, jun_trips, jul_trips, aug_trips, sep_trips, oct_trips,
                       nov_trips, dec_trips], ignore_index=True)

In [8]:
# read taxi_zone_lookup.csv into dataframe
taxi_zones = pd.read_csv("taxi_zone_lookup.csv")

In [12]:
# see what taxi_zones dataframe looks like
taxi_zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [9]:
# service_zone doesn't look very informative, so drop that column
taxi_zones = taxi_zones.drop(columns=["service_zone"])

In [14]:
# see what current taxi_zones dataframe looks like
taxi_zones.head()

,LocationID,Borough,Zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights


In [10]:
# merge to get pickup zones based on location IDs, drop location IDs (since we already have the zone names), and rename columns
all_trips_w_zones = pd.merge(all_trips, taxi_zones, how="left", left_on="PULocationID", right_on="LocationID")
all_trips_w_zones = all_trips_w_zones.drop(columns=["LocationID"])
all_trips_w_zones = all_trips_w_zones.rename(columns={"Borough": "pickup_borough", "Zone": "pickup_zone"})

In [11]:
# see what all_trips_w_zones dataframe looks like
all_trips_w_zones.head()

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,pickup_borough,pickup_zone
0,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,Manhattan,Times Sq/Theatre District
1,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,Manhattan,Manhattanville
2,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,Manhattan,UN/Turtle Bay South
3,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,Manhattan,Lincoln Square East
4,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,Manhattan,Lincoln Square West


In [12]:
# similarly, merge to get dropoff zones based on location IDs, drop location IDs, and rename columns
all_trips_w_zones = pd.merge(all_trips_w_zones, taxi_zones, how="left", left_on="DOLocationID", right_on="LocationID")
all_trips_w_zones = all_trips_w_zones.drop(columns=["LocationID"])
all_trips_w_zones = all_trips_w_zones.rename(columns={"Borough": "dropoff_borough", "Zone": "dropoff_zone"})

In [13]:
# see what current all_trips_w_zones dataframe looks like
all_trips_w_zones.head()

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
0,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,Manhattan,Times Sq/Theatre District,Manhattan,Morningside Heights
1,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,Manhattan,Manhattanville,Bronx,Morrisania/Melrose
2,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,Manhattan,UN/Turtle Bay South,Manhattan,Lincoln Square East
3,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,Manhattan,Lincoln Square East,Manhattan,Lincoln Square West
4,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,Manhattan,Lincoln Square West,Bronx,East Tremont


In [21]:
# see number of distinct values in each column
all_trips_w_zones.nunique()

pickup_datetime     29057434
dropoff_datetime    29033057
pickup_borough             7
pickup_zone              261
dropoff_borough            7
dropoff_zone             261
dtype: int64

In [14]:
# check number of rows in current dataframe
total_num_rows = len(all_trips_w_zones)
print("Number of rows in dataframe:", total_num_rows)

Number of rows in dataframe: 126129064


In [15]:
# check number of rows with unknown boroughs (which would equivalently be those with NV or NA zones)
num_rows_unknown = len(all_trips_w_zones.loc[(all_trips_w_zones["pickup_borough"]=="Unknown") | (all_trips_w_zones["dropoff_borough"]=="Unknown")])
print("Number of rows with unknown boroughs:", num_rows_unknown)
print("Proportion of rows with unknown boroughs:", num_rows_unknown/total_num_rows)

Number of rows with unknown boroughs: 4483145
Proportion of rows with unknown boroughs: 0.03554410742317092


In [16]:
# since proportion of rows with unknown boroughs is small, can just drop them
all_trips_w_zones = all_trips_w_zones.drop(all_trips_w_zones.loc[(all_trips_w_zones["pickup_borough"]=="Unknown") | (all_trips_w_zones["dropoff_borough"]=="Unknown")].index)

In [17]:
all_trips_w_zones.head()

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
0,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,Manhattan,Times Sq/Theatre District,Manhattan,Morningside Heights
1,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,Manhattan,Manhattanville,Bronx,Morrisania/Melrose
2,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,Manhattan,UN/Turtle Bay South,Manhattan,Lincoln Square East
3,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,Manhattan,Lincoln Square East,Manhattan,Lincoln Square West
4,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,Manhattan,Lincoln Square West,Bronx,East Tremont


In [18]:
all_trips_w_zones.loc[(all_trips_w_zones["pickup_borough"]=="Unknown") | (all_trips_w_zones["dropoff_borough"]=="Unknown")]

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone


In [19]:
# will now notice a decrease in number of unique values for boroughs and zones
all_trips_w_zones.nunique()

pickup_datetime     28920572
dropoff_datetime    28891025
PULocationID             261
DOLocationID             262
pickup_borough             6
pickup_zone              260
dropoff_borough            6
dropoff_zone             260
dtype: int64

In [20]:
# check types of dataframe columns
all_trips_w_zones.dtypes

pickup_datetime     datetime64[ns]
dropoff_datetime    datetime64[ns]
PULocationID                 int64
DOLocationID                 int64
pickup_borough              object
pickup_zone                 object
dropoff_borough             object
dropoff_zone                object
dtype: object

In [21]:
# save current dataframe as csv file
compression_opts = dict(method="gzip")
all_trips_w_zones.to_csv("./uber_tripdata_2021_w_ids.csv", index=False, compression=compression_opts)